In [12]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



In [13]:
import json
with open('/Users/NZizi/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [14]:
# import the YelpAPI Class
from yelpapi import YelpAPI
# Create an instance with your key
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api


In [15]:
# set our API call parameters 
LOCATION = 'Greenville, SC'
TERM = 'Sushi'


In [16]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Greenville_SC_sushi.json"
JSON_FILE



'Data/results_in_progress_Greenville_SC_sushi.json'

In [17]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_Greenville_SC_sushi.json not found. Saving empty list to file.


In [18]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [19]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [20]:
## How many results total?
total_results = results['total']
total_results



108

In [21]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

In [22]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



6

In [23]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [24]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 


  0%|          | 0/6 [00:00<?, ?it/s]

In [25]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

  ## If it DOES exist:
    if file_exists == True:
        if delete_if_exists==True:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")
    else:
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        if len(folder)>0:
            os.makedirs(folder,exist_ok=True)

        with open(JSON_FILE,'w') as f:
            json.dump([],f)
                  
            
    



In [26]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[!] Data/results_in_progress_Greenville_SC_sushi.json already exists. Deleting previous file...
[i] Data/results_in_progress_Greenville_SC_sushi.json not found. Saving empty list to new file.
- 0 previous results found.


6

In [27]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/6 [00:00<?, ?it/s]

In [28]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,RGRk1ioORwm_FIX8PM732Q,konnichiwa-greenville,Konnichiwa,https://s3-media3.fl.yelpcdn.com/bphoto/p47H0_...,False,https://www.yelp.com/biz/konnichiwa-greenville...,62,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 34.845952342825115, 'longitude': ...",[],"{'address1': '101 Falls Park Dr', 'address2': ...",+18642524436,(864) 252-4436,4184.255183,NaN
1,zG_XOAFi9Y560WJ1RvghBw,sushi-masa-japanese-restaurant-greenville,Sushi-Masa Japanese Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/zsRavZ...,False,https://www.yelp.com/biz/sushi-masa-japanese-r...,160,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.5,"{'latitude': 34.8512725830078, 'longitude': -8...",[delivery],"{'address1': '8590 Pelham Rd', 'address2': 'St...",+18642882227,(864) 288-2227,11481.830881,$$
2,babpQQHQSGjVxS1Li7Q2lA,otto-izakaya-downtown-greenville,Otto Izakaya - Downtown,https://s3-media2.fl.yelpcdn.com/bphoto/CQ7zdN...,False,https://www.yelp.com/biz/otto-izakaya-downtown...,180,"[{'alias': 'izakaya', 'title': 'Izakaya'}, {'a...",3.5,"{'latitude': 34.84428, 'longitude': -82.406}","[restaurant_reservation, delivery]","{'address1': '802 S Main St', 'address2': '', ...",+18645685880,(864) 568-5880,4427.139837,$$
3,7cJxOV-ANX1qLThK3yV96w,otto-izakaya-greenville-4,Otto Izakaya,https://s3-media1.fl.yelpcdn.com/bphoto/TdPhFy...,False,https://www.yelp.com/biz/otto-izakaya-greenvil...,441,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 34.8228218820722, 'longitude': -8...",[delivery],"{'address1': '15 Market Point Dr', 'address2':...",+18645688009,(864) 568-8009,5933.485357,$$
4,Gx1IhUJPwgTVQcJmw1kD0A,sushi-kingdom-greenville-2,Sushi Kingdom,https://s3-media4.fl.yelpcdn.com/bphoto/5zeS32...,False,https://www.yelp.com/biz/sushi-kingdom-greenvi...,78,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 34.83118194997925, 'longitude': -...","[pickup, delivery]","{'address1': '1117 Woodruff Rd', 'address2': '...",+18642438313,(864) 243-8313,5189.593817,NaN


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
103,xb9QSdbk63Ani2-S5MrIHQ,harris-teeter-greenville-6,Harris Teeter,https://s3-media3.fl.yelpcdn.com/bphoto/ZelRSg...,False,https://www.yelp.com/biz/harris-teeter-greenvi...,27,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",3.5,"{'latitude': 34.8279736, 'longitude': -82.3987...",[],"{'address1': '1720 Augusta St', 'address2': ''...",+18649778041,(864) 977-8041,4335.688854,$$
104,Dz1TrhAHtAiCE8ySrQV4vA,publix-super-markets-greenville-2,Publix Super Markets,https://s3-media3.fl.yelpcdn.com/bphoto/GQkkow...,False,https://www.yelp.com/biz/publix-super-markets-...,22,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.5,"{'latitude': 34.86810664795167, 'longitude': -...",[],"{'address1': '215 Pelham Rd', 'address2': '', ...",+18643708210,(864) 370-8210,2477.641189,$$
105,xQZIvcjkH2R14yaHr2qQYQ,the-cheesecake-factory-greenville-2,The Cheesecake Factory,https://s3-media4.fl.yelpcdn.com/bphoto/3JnqQd...,False,https://www.yelp.com/biz/the-cheesecake-factor...,462,"[{'alias': 'desserts', 'title': 'Desserts'}, {...",3.0,"{'latitude': 34.8499166, 'longitude': -82.3335...",[delivery],"{'address1': '700 Haywood Mall', 'address2': '...",+18642884444,(864) 288-4444,2209.333296,$$
106,zTTrMt6nvB_bw0j3RUEsLA,new-china-greer,New China,https://s3-media2.fl.yelpcdn.com/bphoto/IGpt7F...,False,https://www.yelp.com/biz/new-china-greer?adjus...,17,"[{'alias': 'chinese', 'title': 'Chinese'}]",2.5,"{'latitude': 34.9476509, 'longitude': -82.2240...",[delivery],"{'address1': '614 N Main St', 'address2': '', ...",+18648778885,(864) 877-8885,16504.999220,$
107,_BLlWxSpx1mRGW9eFutYdQ,dairy-queen-grill-and-chill-mauldin-2,Dairy Queen Grill & Chill,https://s3-media1.fl.yelpcdn.com/bphoto/SMirq1...,False,https://www.yelp.com/biz/dairy-queen-grill-and...,22,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.0,"{'latitude': 34.780454197801, 'longitude': -82...","[pickup, delivery]","{'address1': '112 N Main St', 'address2': None...",+18643739896,(864) 373-9896,8623.432240,$


In [29]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()



0

In [30]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()


0

In [31]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Greenville_SC_sushi.csv.gz', compression='gzip',index=False)
